[View in Colaboratory](https://colab.research.google.com/github/lmegha/AdminLTE/blob/master/test.ipynb)

In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
!ls

datalab


In [8]:
!git clone https://github.com/cchen156/Learning-to-See-in-the-Dark.git

fatal: destination path 'Learning-to-See-in-the-Dark' already exists and is not an empty directory.


In [11]:
import requests
import os

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



print('Dowloading Sony subset... (25GB)')
download_file_from_google_drive('10kpAcvldtcb9G2ze5hTcF1odzu4V_Zvh', 'Learning-to-See-in-the-Dark/dataset/Sony.zip')

print('Downloading Fuji subset... (52GB)')
download_file_from_google_drive('12hvKCjwuilKTZPe9EZ7ZTb-azOmUA3HT', 'Learning-to-See-in-the-Dark/dataset/Fuji.zip')

os.system('unzip Learning-to-See-in-the-Dark/dataset/Sony.zip')
os.system('unzip Learning-to-See-in-the-Dark/dataset/Fuji.zip')
print('Unzipped Files')

Dowloading Sony subset... (25GB)
Unzipped Files


In [12]:
!ls

datalab		     Fuji_val_list.txt		  Sony_train_list.txt
Fuji		     Learning-to-See-in-the-Dark  Sony_val_list.txt
Fuji_test_list.txt   Sony
Fuji_train_list.txt  Sony_test_list.txt


In [13]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



print('Dowloading Sony Model (84Mb)')
download_file_from_google_drive('1wmx7AM6XWHjHIvpErmIouQgbQoMxAymG', 'Learning-to-See-in-the-Dark/checkpoint/Sony/model.ckpt.data-00000-of-00001')
download_file_from_google_drive('1OmrGMng1QuwUa8lf-_wBVvbRJwBr0ETr', 'Learning-to-See-in-the-Dark/checkpoint/Sony/model.ckpt.meta')

print('Dowloading Fuji Model (84Mb)')
download_file_from_google_drive('1PX5wA89d-JLmwQHqpBnyTYJxDVzC1gpt', 'Learning-to-See-in-the-Dark/checkpoint/Fuji/model.ckpt.data-00000-of-00001')
download_file_from_google_drive('1VzyzQ9JglcxxqUe8yn3-cAeB1pJ4jxf4', 'Learning-to-See-in-the-Dark/checkpoint/Fuji/model.ckpt.meta')

Dowloading Sony Model (84Mb)
Dowloading Fuji Model (84Mb)


In [14]:
!ls

datalab		     Fuji_val_list.txt		  Sony_train_list.txt
Fuji		     Learning-to-See-in-the-Dark  Sony_val_list.txt
Fuji_test_list.txt   Sony
Fuji_train_list.txt  Sony_test_list.txt


In [15]:
!pip install rawpy

    100% |████████████████████████████████| 624kB 10.8MB/s 


In [0]:
tf.reset_default_graph()

In [16]:
#uniform content loss + adaptive threshold + per_class_input + recursive G
#improvement upon cqf37
from __future__ import division
import os,time,scipy.io
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers import initializers
import numpy as np
import pdb
import rawpy
import glob


input_dir = './Sony/short/'
gt_dir = './Sony/long/'
checkpoint_dir = './Learning-to-See-in-the-Dark/checkpoint/Sony/'
result_dir = './result_Sony/'

#get train and test IDs
train_fns = glob.glob(gt_dir + '0*.ARW')
train_ids = []
for i in range(len(train_fns)):
    _, train_fn = os.path.split(train_fns[i])
    train_ids.append(int(train_fn[0:5]))

test_fns = glob.glob(gt_dir + '/1*.ARW')
test_ids = []
for i in range(len(test_fns)):
    _, test_fn = os.path.split(test_fns[i])
    test_ids.append(int(test_fn[0:5]))



ps = 512 #patch size for training
save_freq = 500

DEBUG = 0
if DEBUG == 1:
  save_freq = 2
  train_ids = train_ids[0:5]
  test_ids = test_ids[0:5]



def lrelu(x):
    return tf.maximum(x*0.2,x)

def upsample_and_concat(x1, x2, output_channels, in_channels):

    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal( [pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2) , strides=[1, pool_size, pool_size, 1] )

    deconv_output =  tf.concat([deconv, x2],3)
    deconv_output.set_shape([None, None, None, output_channels*2])

    return deconv_output

def network(input):
    conv1=slim.conv2d(input,32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv1_1')
    conv1=slim.conv2d(conv1,32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv1_2')
    pool1=slim.max_pool2d(conv1, [2, 2], padding='SAME' )

    conv2=slim.conv2d(pool1,64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv2_1')
    conv2=slim.conv2d(conv2,64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv2_2')
    pool2=slim.max_pool2d(conv2, [2, 2], padding='SAME' )

    conv3=slim.conv2d(pool2,128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv3_1')
    conv3=slim.conv2d(conv3,128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv3_2')
    pool3=slim.max_pool2d(conv3, [2, 2], padding='SAME' )

    conv4=slim.conv2d(pool3,256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv4_1')
    conv4=slim.conv2d(conv4,256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv4_2')
    pool4=slim.max_pool2d(conv4, [2, 2], padding='SAME' )

    conv5=slim.conv2d(pool4,512,[3,3], rate=1, activation_fn=lrelu,scope='g_conv5_1')
    conv5=slim.conv2d(conv5,512,[3,3], rate=1, activation_fn=lrelu,scope='g_conv5_2')

    up6 =  upsample_and_concat( conv5, conv4, 256, 512  )
    conv6=slim.conv2d(up6,  256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv6_1')
    conv6=slim.conv2d(conv6,256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv6_2')

    up7 =  upsample_and_concat( conv6, conv3, 128, 256  )
    conv7=slim.conv2d(up7,  128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv7_1')
    conv7=slim.conv2d(conv7,128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv7_2')

    up8 =  upsample_and_concat( conv7, conv2, 64, 128 )
    conv8=slim.conv2d(up8,  64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv8_1')
    conv8=slim.conv2d(conv8,64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv8_2')

    up9 =  upsample_and_concat( conv8, conv1, 32, 64 )
    conv9=slim.conv2d(up9,  32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv9_1')
    conv9=slim.conv2d(conv9,32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv9_2')

    conv10=slim.conv2d(conv9,12,[1,1], rate=1, activation_fn=None, scope='g_conv10')
    out = tf.depth_to_space(conv10,2)
    return out


def pack_raw(raw):
    #pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32) 
    im = np.maximum(im - 512,0)/ (16383 - 512) #subtract the black level

    im = np.expand_dims(im,axis=2) 
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2,0:W:2,:], 
                       im[0:H:2,1:W:2,:],
                       im[1:H:2,1:W:2,:],
                       im[1:H:2,0:W:2,:]), axis=2)
    return out



sess=tf.Session()
in_image=tf.placeholder(tf.float32,[None,None,None,4])
gt_image=tf.placeholder(tf.float32,[None,None,None,3])
out_image=network(in_image)

saver=tf.train.Saver()
sess.run(tf.global_variables_initializer())
ckpt=tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt:
    print('loaded '+ckpt.model_checkpoint_path)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    
if not os.path.isdir(result_dir + 'final/'):
    os.makedirs(result_dir + 'final/')

for test_id in test_ids:
    #test the first image in each sequence
    in_files = glob.glob(input_dir + '%05d_00*.ARW'%test_id)
    for k in range(len(in_files)):
        in_path = in_files[k]
        _, in_fn = os.path.split(in_path)
        print(in_fn)
        gt_files = glob.glob(gt_dir + '%05d_00*.ARW'%test_id) 
        gt_path = gt_files[0]
        _, gt_fn = os.path.split(gt_path)
        in_exposure =  float(in_fn[9:-5])
        gt_exposure =  float(gt_fn[9:-5])
        ratio = min(gt_exposure/in_exposure,300)

        raw = rawpy.imread(in_path)
        input_full = np.expand_dims(pack_raw(raw),axis=0) *ratio
        
        im = raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        #scale_full = np.expand_dims(np.float32(im/65535.0),axis = 0)*ratio
        scale_full = np.expand_dims(np.float32(im/65535.0),axis = 0)	

        gt_raw = rawpy.imread(gt_path)
        im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        gt_full = np.expand_dims(np.float32(im/65535.0),axis = 0)

        input_full = np.minimum(input_full,1.0)
         
        output =sess.run(out_image,feed_dict={in_image: input_full})
        output = np.minimum(np.maximum(output,0),1)

        output = output[0,:,:,:]
        gt_full = gt_full[0,:,:,:]
        scale_full = scale_full[0,:,:,:] 	
        scale_full = scale_full*np.mean(gt_full)/np.mean(scale_full) # scale the low-light image to the same mean of the groundtruth
        
        scipy.misc.toimage(output*255,  high=255, low=0, cmin=0, cmax=255).save(result_dir + 'final/%5d_00_%d_out.png'%(test_id,ratio))
        scipy.misc.toimage(scale_full*255,  high=255, low=0, cmin=0, cmax=255).save(result_dir + 'final/%5d_00_%d_scale.png'%(test_id,ratio))
        scipy.misc.toimage(gt_full*255,  high=255, low=0, cmin=0, cmax=255).save(result_dir + 'final/%5d_00_%d_gt.png'%(test_id,ratio))


loaded ./Learning-to-See-in-the-Dark/checkpoint/Sony/model.ckpt
INFO:tensorflow:Restoring parameters from ./Learning-to-See-in-the-Dark/checkpoint/Sony/model.ckpt
10069_00_0.1s.ARW
10069_00_0.04s.ARW
10187_00_0.04s.ARW
10187_00_0.1s.ARW
10187_00_0.033s.ARW
10054_00_0.1s.ARW
10054_00_0.04s.ARW
10203_00_0.033s.ARW
10203_00_0.04s.ARW
10203_00_0.1s.ARW
10185_00_0.04s.ARW
10185_00_0.033s.ARW
10185_00_0.1s.ARW
10162_00_0.1s.ARW
10178_00_0.033s.ARW
10178_00_0.1s.ARW
10178_00_0.04s.ARW
10125_00_0.1s.ARW
10176_00_0.1s.ARW
10074_00_0.1s.ARW
10055_00_0.1s.ARW
10055_00_0.04s.ARW
10198_00_0.04s.ARW
10198_00_0.033s.ARW
10198_00_0.1s.ARW
10003_00_0.1s.ARW
10003_00_0.04s.ARW
10011_00_0.04s.ARW
10011_00_0.1s.ARW
10040_00_0.1s.ARW
10040_00_0.04s.ARW
10126_00_0.1s.ARW
10035_00_0.04s.ARW
10035_00_0.1s.ARW
10213_00_0.04s.ARW
10213_00_0.033s.ARW
10213_00_0.1s.ARW
10191_00_0.033s.ARW
10191_00_0.1s.ARW
10191_00_0.04s.ARW
10034_00_0.1s.ARW
10034_00_0.04s.ARW
10022_00_0.04s.ARW
10022_00_0.1s.ARW
10101_00_0.1s.A

In [17]:
files = !ls result_Sony | head
files

['final']

In [1]:
!ls {PATH}

ls: cannot access '{PATH}': No such file or directory


In [0]:
from google.colab import files
files.download('results.zip')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 37288, 0, 0)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 654, in __init__
    self.finish()
  File "/usr/lib/python2.7/SocketServer.py", line 713, in finish
    self.wfile.close()
  File "/usr/lib/python2.7/socket.py", line 283, in close
    self.flush()
  File "/usr/lib/python2.7/socket.py", line 307, in flush
    self._sock.sendall(view[write_offset:write_offset+buffer_size])
error: [Errno 32] Broken pipe


In [2]:
!ls

datalab		     Fuji_val_list.txt		  Sony
Fuji		     Learning-to-See-in-the-Dark  Sony_test_list.txt
Fuji_test_list.txt   result_Sony		  Sony_train_list.txt
Fuji_train_list.txt  results.zip		  Sony_val_list.txt


In [1]:
!zip -r results.zip result_Sony

  adding: result_Sony/ (stored 0%)
  adding: result_Sony/final/ (stored 0%)
  adding: result_Sony/final/10192_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10163_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10101_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10103_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10022_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10125_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10217_00_100_gt.png (deflated 0%)
  adding: result_Sony/final/10178_00_100_scale.png (deflated 0%)
  adding: result_Sony/final/10217_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10178_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10003_00_100_scale.png (deflated 0%)
  adding: result_Sony/final/10167_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10011_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10125_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10035

 (deflated 0%)
  adding: result_Sony/final/10193_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10198_00_100_scale.png (deflated 0%)
  adding: result_Sony/final/10074_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10217_00_250_out.png (deflated 0%)
  adding: result_Sony/final/10140_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10069_00_250_scale.png (deflated 0%)
  adding: result_Sony/final/10170_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10074_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10227_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10040_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10185_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10199_00_100_gt.png (deflated 0%)
  adding: result_Sony/final/10176_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10162_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10055_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10105_00_3

 (deflated 0%)
  adding: result_Sony/final/10178_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10140_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10034_00_250_out.png (deflated 0%)
  adding: result_Sony/final/10126_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10068_00_100_gt.png (deflated 0%)
  adding: result_Sony/final/10011_00_100_gt.png (deflated 0%)
  adding: result_Sony/final/10034_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10203_00_250_scale.png (deflated 0%)
  adding: result_Sony/final/10176_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10187_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10022_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10016_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10055_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10139_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10106_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10093_00_300_o

 (deflated 0%)
  adding: result_Sony/final/10191_00_100_scale.png (deflated 0%)
  adding: result_Sony/final/10040_00_100_scale.png (deflated 0%)
  adding: result_Sony/final/10203_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10035_00_250_out.png (deflated 0%)
  adding: result_Sony/final/10103_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10069_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10022_00_250_scale.png (deflated 0%)
  adding: result_Sony/final/10193_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10068_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10187_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10203_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10101_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10077_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10167_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10006_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10054_00_100_

 (deflated 0%)
  adding: result_Sony/final/10030_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10193_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10226_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10111_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10016_00_100_gt.png (deflated 0%)
  adding: result_Sony/final/10055_00_100_gt.png (deflated 0%)
  adding: result_Sony/final/10178_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10213_00_250_out.png (deflated 0%)
  adding: result_Sony/final/10035_00_100_gt.png (deflated 0%)
  adding: result_Sony/final/10172_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10016_00_100_scale.png (deflated 0%)
  adding: result_Sony/final/10054_00_100_scale.png (deflated 0%)
  adding: result_Sony/final/10228_00_100_out.png (deflated 0%)
  adding: result_Sony/final/10162_00_300_gt.png (deflated 0%)
  adding: result_Sony/final/10213_00_250_scale.png (deflated 0%)
  adding: result_Sony/final/10192_00_10

 (deflated 0%)
  adding: result_Sony/final/10011_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10040_00_250_out.png (deflated 0%)
  adding: result_Sony/final/10082_00_300_out.png (deflated 0%)
  adding: result_Sony/final/10040_00_250_gt.png (deflated 0%)
  adding: result_Sony/final/10167_00_300_scale.png (deflated 0%)
  adding: result_Sony/final/10178_00_250_out.png (deflated 0%)


In [0]:
import cv2
cv2.imshow('final',cv2.imread('result_Sony/final/10011_00_250_out.png'))